1. Import library

In [2]:
from bs4 import BeautifulSoup
from pymongo import MongoClient
import requests
import json

2. Crawl data

In [4]:
#hoanghamobile.com
url = "https://hoanghamobile.com/"

In [5]:
#hàm lấy html qua 1 link url 
def access(link):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
#test 
access(url)


<!DOCTYPE html>

<html>
<head>
<meta content="width=device-width" name="viewport"/>
<link href="https://fonts.googleapis.com/css2?family=Inter:wght@100..900&amp;display=swap" rel="stylesheet"/>
<link href="/Content/web/fonts/icomoon-v1.0/style.css?v=1.3.2.7" rel="stylesheet">
<link href="/Content/web/js/plugins/jquery-slick-1.8.1/slick.css?v=1.3.2.7" rel="stylesheet"/>
<link href="/Content/web/js/plugins/jquery-slick-1.8.1/slick-theme.css?v=1.3.2.7" rel="stylesheet"/>
<link href="/Content/web_detail/css/formOrder.css?v=1.3.2.7" rel="stylesheet"/>
<link href="/Content/V4.Electric/web/css/v4.index.css?v=1.3.2.7" rel="stylesheet"/>
<link href="/Content/web/js/plugins/OwlCarousel2-2.3.4/assets/owl.carousel.min.css?v=1.3.2.7" rel="stylesheet" type="text/css"/>
<link href="/Content/web/js/plugins/jquery.modal/jquery.modal.min.css?v=1.3.2.7" rel="stylesheet">
<link href="/Content/web/js/plugins/jquery.modal/jquery.modal.custom.css?v=1.3.2.7" rel="stylesheet"/>
<link href="/Content/web/js/plu

In [33]:
#lấy link theo danh mục 
html_root = access(url)
def get_category_links_list(html):
    category_tag = html.find('div', 'nav-full')
    category_list = category_tag.find_all('li')
    category_links_list = ['https://hoanghamobile.com' + category.find('a').get('href') for category in category_list]
    
    return category_links_list

#test
category_links_list = get_category_links_list(html_root)
#lay link cho dien may vi trong dien may con chia catrgories tiep
link_electonics = get_category_links_list(access(category_links_list[5]))
#xoa apple va cac dich vu dang sau
category_links_list = category_links_list[:1] + category_links_list[2:5] + category_links_list[6:7] + link_electonics
print(category_links_list)

['https://hoanghamobile.com/dien-thoai-di-dong', 'https://hoanghamobile.com/laptop', 'https://hoanghamobile.com/tablet', 'https://hoanghamobile.com/man-hinh', 'https://hoanghamobile.com/dong-ho', 'https://hoanghamobile.com/smart-tv', 'https://hoanghamobile.com/dien-may/dien-lanh/tu-lanh', 'https://hoanghamobile.com/dien-may/dien-lanh/dieu-hoa', 'https://hoanghamobile.com/dien-may/dien-lanh/may-giat', 'https://hoanghamobile.com/dien-may/loa-soundbar', 'https://hoanghamobile.com/dien-may/do-gia-dung', 'https://hoanghamobile.com/dien-may/dien-tu', 'https://hoanghamobile.com/dien-may/dien-lanh']


In [38]:
#lay link cac san pham tu theo brand
def get_items_links_by_brands_list(category_link):
    #phan loai theo nhan hang
    html = access(category_link)
    brands_tag = html.find('div', 'v5-brand-list')
    
    if brands_tag == None:
        return [category_link] #có 1 số trang không thể chia theo brand
    
    brands_a_tag = brands_tag.find_all('a')
    brand_links = ['https://hoanghamobile.com' + brand_a_tag.get('href') for brand_a_tag in brands_a_tag]
    return brand_links



#test
ex_link = 'https://hoanghamobile.com/dien-thoai-di-dong'
print(get_items_links_by_brands_list(ex_link))

['https://hoanghamobile.com/dien-thoai-di-dong/iphone', 'https://hoanghamobile.com/dien-thoai-di-dong/samsung', 'https://hoanghamobile.com/dien-thoai-di-dong/xiaomi', 'https://hoanghamobile.com/dien-thoai-di-dong/tecno', 'https://hoanghamobile.com/dien-thoai-di-dong/oppo', 'https://hoanghamobile.com/dien-thoai-di-dong/honor', 'https://hoanghamobile.com/dien-thoai-di-dong/oscal', 'https://hoanghamobile.com/dien-thoai-di-dong/redmagic', 'https://hoanghamobile.com/dien-thoai-di-dong/vivo', 'https://hoanghamobile.com/dien-thoai-di-dong/nokia', 'https://hoanghamobile.com/dien-thoai-di-dong/xphone', 'https://hoanghamobile.com/dien-thoai-di-dong/infinix', 'https://hoanghamobile.com/dien-thoai-di-dong/htc', 'https://hoanghamobile.com/dien-thoai-di-dong/inoi', 'https://hoanghamobile.com/dien-thoai-di-dong/realme', 'https://hoanghamobile.com/dien-thoai-di-dong/nubia', 'https://hoanghamobile.com/dien-thoai-di-dong/xor', 'https://hoanghamobile.com/dien-thoai-di-dong/masstel', 'https://hoanghamobil

In [ ]:
#lay thong tin san pham
def get_item_information(link):
    html = access(link)
    

In [56]:
#lay link thong tin san pham tu link brand
def get_item_links_list(brand_link):
    #vao cac trang
    in4 = []
    page_link = brand_link + '?p=1'
    while(1):
        html = access(page_link)
        item_tags = html.find_all('div', 'v5-item')
        item_links = ['https://hoanghamobile.com' + item_tag.find('a').get('href') for item_tag in item_tags]
        print(item_links)
        
        
        more_page_tag = html.find('div', 'v5-more-product')
        if  more_page_tag == None:
            break
        else :
            more_page_a_tag = more_page_tag.find('a')
            page_link =  'https://hoanghamobile.com' + more_page_a_tag.get('href')

        
        
    
 


#test 
print(get_item_links_list('https://hoanghamobile.com/smart-tv/tivi-tcl'))

['https://hoanghamobile.com/tivi/tcl-smart-tivi-32s5400a', 'https://hoanghamobile.com/tivi/tcl-smart-tivi-40s5400a', 'https://hoanghamobile.com/tivi/tcl-smart-tivi-43s5400a', 'https://hoanghamobile.com/tivi/tcl-google-tivi-55p638', 'https://hoanghamobile.com/tivi/tcl-google-tivi-43p755', 'https://hoanghamobile.com/tivi/tcl-google-tivi-55p755-pro', 'https://hoanghamobile.com/tivi/tcl-google-tivi-65p755-pro', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-43c655', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-55c655', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-65c655', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-75c655', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qd-mini-led-55c755']
['https://hoanghamobile.com/tivi/tcl-google-tivi-qd-mini-led-65c755', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qd-mini-led-115x955max', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-85c655', 'https://hoanghamobile.com/tivi/tcl-google-tivi-qled-98c655', 'http